# 加载试用 qwen3-embedding 模型

In [1]:
from sentence_transformers import SentenceTransformer

# Load the model
model_name = "Qwen/Qwen3-Embedding-0.6B"
model_path = "G:\models"
model = SentenceTransformer(model_path)

OSError: [WinError 1920] 系统无法访问此文件。 Error loading "d:\Miniconda3\envs\raft\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [2]:
# The queries and documents to embed
queries = [
    "What is the capital of China?",
    "Explain gravity",
]
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)

d:\Miniconda3\envs\raft\lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


tensor([[0.7646, 0.1414],
        [0.1355, 0.6000]])


In [ ]:
from sentence_transformers import losses
train_loss = losses.CosineSimilarityLoss()


In [2]:
model = SentenceTransformer('G:/models')
model.save('G:/models_repaired')  # This generates all required files
repaired_model = SentenceTransformer('G:/models_repaired')

2025-06-16 09:57:53 - Use pytorch device_name: cuda:0
2025-06-16 09:57:53 - Load pretrained SentenceTransformer: G:/models


TypeError: argument of type 'NoneType' is not iterable

In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
import logging
import json
from torch.utils.data import DataLoader

# 
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    # filename='training.log',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
# 
train_examples = []
with open('data_emb/train01.jsonl', "r") as file:
    for line in file:
        data = json.loads(line)
        query = data['query']
        pos = data['pos'][0]  # 假设只取第一个正样本
        negs = data['neg']
        for neg in negs:
            train_examples.append(InputExample(texts=[query, pos, neg]))
print(f"length of train_examples: {len(train_examples)}")
# 
dev_queries = []
dev_positives = []
dev_negatives = []
with open('data_emb/dev01.jsonl', "r") as file:
    for line in file:
        data = json.loads(line)
        query = data['query']
        pos = data['pos'][0]  # 假设只取第一个正样本
        negs = data['neg']
        
        dev_queries.append(query)
        dev_positives.append(pos)
        dev_negatives.append(negs[0])
print(f"length of dev_examples: {len(dev_queries)}")
dev_evaluator = TripletEvaluator(
    dev_queries,
    dev_positives,
    dev_negatives,
    name="dev",
    batch_size=16
)
# 
model_path = r"G:\models"
# model = SentenceTransformer(model_path)
model = SentenceTransformer(model_path, model_kwargs={"prompts": None})
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# 定义损失
train_loss = losses.TripletLoss(model=model)
# 开始微调
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=2,
    show_progress_bar=True,
    output_path='./model/qwen_triplet_model',
    evaluator=dev_evaluator,
    evaluation_steps=4,
    checkpoint_path="./model/checkpoints",
    checkpoint_save_steps=4,
    checkpoint_save_total_limit=2,
)
print('微调完成')

d:\Miniconda3\envs\qwen_emb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


length of train_examples: 31770
length of dev_examples: 264
2025-06-16 09:56:01 - Use pytorch device_name: cuda:0
2025-06-16 09:56:01 - Load pretrained SentenceTransformer: G:\models


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
import json
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 加载原始模型 
baselinemodel_path = "G:\models"
model_baseline = SentenceTransformer(baselinemodel_path)
# 加载微调后的模型
model_finetuned = SentenceTransformer('./model/qwen_triplet_model')

def evaluate_model(model, data_path):
    queries = []
    candidates = []
    labels = []
    with open(data_path, 'r', encoding='utf-8') as file:
        for line in file:
            item = json.loads(line.strip())
            query = item['query']
            pos = item['pos'][0]
            negs = item["negs"]
            
            queries.append(query)
            candidates.append(pos)
            labels.append(len(candidates) - 1) # 正样本的位置 
            for neg in negs:
                candidates.append(neg)
    # 编码查询和候选文档
    query_embeddings = model.encode(queries, convert_to_tensor=True)
    candidate_embeddings = model.encode(candidates, convert_to_tensor=True)
    # 计算相似度矩阵
    scores = cosine_similarity(query_embeddings.cpu(), candidate_embeddings.cpu())
    # 排序，并计算 Recall@10, MRR@10
    recall_at_10, mrr_at_10 = 0, 0
    for i in range(len(queries)):
        ranked_indices = np.argsort(-scores[i])[:10] # 取top10
        for labels[i] in ranked_indices:
            recall_at_10 += 1
            rank = np.where(ranked_indices == labels[i])[0][0] + 1
            mrr_at_10 += 1.0 / rank
    recall_at_10 /= len(queries)
    mrr_at_10 /= len(queries)
    return recall_at_10, mrr_at_10

# 测试 baseline 模型
print(f"Evaluating baseline model...")
recall_base, mrr_base = evaluate_model(model_baseline, 'data_emb/dev.jsonl')
print(f"Baseline: Recall@10: {recall_base:.4f}, MRR@10: {mrr_base:.4f}")

# 测试 fine-tuned 模型
print(f"Evaluating fine-tuned model...")
recall_ft, mrr_ft = evaluate_model(model_finetuned, 'data_emb/dev.jsonl')
print(f"Fine-tuned: Recall@10: {recall_ft:.4f}, MRR@10: {mrr_ft:.4f}")

In [1]:
import json
import random
import os

# 设置路径
input_file = 'data_emb/output62hn.jsonl'
output_dir = 'data_emb'

os.makedirs(output_dir, exist_ok=True)

train_file = os.path.join(output_dir, 'train01.jsonl')
dev_file = os.path.join(output_dir, 'dev01.jsonl')
test_file = os.path.join(output_dir, 'test01.jsonl')

# 拆分比例：80% 训练，10% 验证，10% 测试
train_ratio = 0.8
dev_ratio = 0.1
test_ratio = 0.1

# 读取所有数据
with open(input_file, 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]

# 打乱顺序确保分布均匀
random.shuffle(lines)

# 计算分割点
total = len(lines)
train_end = int(total * train_ratio)
dev_end = train_end + int(total * dev_ratio)

train_data = lines[:train_end]
dev_data = lines[train_end:dev_end]
test_data = lines[dev_end:]

# 写入文件
def write_lines(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        for line in data:
            f.write(line + '\n')

write_lines(train_file, train_data)
write_lines(dev_file, dev_data)
write_lines(test_file, test_data)

print(f"Total samples: {total}")
print(f"Train: {len(train_data)}")
print(f"Dev:   {len(dev_data)}")
print(f"Test:  {len(test_data)}")
print("Split complete.")

Total samples: 2648
Train: 2118
Dev:   264
Test:  266
Split complete.


In [4]:
from sentence_transformers import SentenceTransformer

# 1. 加载已有模型（使用目录下存在的实际文件名）
model = SentenceTransformer('G:/models')  # 注意使用正斜杠路径

# 2. 重新保存生成标准配置文件
model.save('G:/models_repaired')  # 自动生成所有正确文件

# 3. 后续使用新目录
repaired_model = SentenceTransformer('G:/models_repaired')

2025-06-16 09:44:07 - Use pytorch device_name: cuda:0
2025-06-16 09:44:07 - Load pretrained SentenceTransformer: G:/models


FileNotFoundError: [Errno 2] No such file or directory: 'G:/models\\sentence_xlnet_config.json'